# ಸೆಷನ್ 2 – ರಾಗಗಳೊಂದಿಗೆ RAG ಮೌಲ್ಯಮಾಪನ

ರಾಗಗಳ ಮೆಟ್ರಿಕ್ಸ್ ಬಳಸಿ ಕನಿಷ್ಠ RAG ಪೈಪ್ಲೈನ್ ಅನ್ನು ಮೌಲ್ಯಮಾಪನ ಮಾಡಿ: answer_relevancy, faithfulness, context_precision.


# ದೃಶ್ಯ
ಈ ದೃಶ್ಯವು ಸ್ಥಳೀಯವಾಗಿ ಕನಿಷ್ಠ Retrieval Augmented Generation (RAG) ಪೈಪ್ಲೈನ್ ಅನ್ನು ಮೌಲ್ಯಮಾಪನ ಮಾಡುತ್ತದೆ. ನಾವು:
- ಒಂದು ಸಣ್ಣ ಸಿಂಥೆಟಿಕ್ ಡಾಕ್ಯುಮೆಂಟ್ ಕಾರ್ಪಸ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೇವೆ.
- ಡಾಕ್ಸ್ ಅನ್ನು ಎम्बೆಡ್ ಮಾಡಿ ಮತ್ತು ಸರಳ ಸಾದೃಶ್ಯ ರಿಟ್ರೀವರ್ ಅನ್ನು ಜಾರಿಗೆ ತರುತ್ತೇವೆ.
- ಸ್ಥಳೀಯ ಮಾದರಿಯನ್ನು (Foundry Local / OpenAI-ಸಮ್ಮತ) ಬಳಸಿ ನೆಲಸಿದ ಉತ್ತರಗಳನ್ನು ರಚಿಸುತ್ತೇವೆ.
- ragas ಮೆಟ್ರಿಕ್ಸ್ (`answer_relevancy`, `faithfulness`, `context_precision`) ಅನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ.
- ವೇಗದ ಮೋಡ್ (ಪರಿಸರ `RAG_FAST=1`) ಅನ್ನು ಬೆಂಬಲಿಸುತ್ತೇವೆ, ಇದು ತ್ವರಿತ ಪುನರಾವೃತ್ತಿಗಾಗಿ ಕೇವಲ ಉತ್ತರ ಸಂಬಂಧಿತತೆಯನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತದೆ.

ನಿಮ್ಮ ಸ್ಥಳೀಯ ಮಾದರಿ + ಎम्बೆಡಿಂಗ್ಸ್ ಸ್ಟ್ಯಾಕ್ ವಿಸ್ತೃತ ಕಾರ್ಪಸ್‌ಗಳಿಗೆ ವಿಸ್ತರಿಸುವ ಮೊದಲು ವಾಸ್ತವಿಕವಾಗಿ ನೆಲಸಿದ ಉತ್ತರಗಳನ್ನು ಉತ್ಪಾದಿಸುತ್ತಿದೆಯೇ ಎಂದು ಪರಿಶೀಲಿಸಲು ಈ ನೋಟ್ಬುಕ್ ಅನ್ನು ಬಳಸಿ.


### ವಿವರಣೆ: ಅವಲಂಬನೆ ಸ್ಥಾಪನೆ
ಅಗತ್ಯವಾದ ಗ್ರಂಥಾಲಯಗಳನ್ನು ಸ್ಥಾಪಿಸುತ್ತದೆ:
- ಸ್ಥಳೀಯ ಮಾದರಿ ನಿರ್ವಹಣೆಗೆ `foundry-local-sdk`.
- `openai` ಕ್ಲೈಂಟ್ ಇಂಟರ್ಫೇಸ್.
- ದಟ್ಟವಾದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳಿಗೆ `sentence-transformers`.
- ಮೌಲ್ಯಮಾಪನ ಮತ್ತು ಮೆಟ್ರಿಕ್ ಗಣನೆಗೆ `ragas` + `datasets`.
- ragas LLM ಇಂಟರ್ಫೇಸ್‌ಗಾಗಿ `langchain-openai` ಅಡಾಪ್ಟರ್.

ಪುನಃ ಚಾಲನೆ ಮಾಡಲು ಸುರಕ್ಷಿತ; ಪರಿಸರವು ಈಗಾಗಲೇ ಸಿದ್ಧವಾಗಿದ್ದರೆ ಬಿಟ್ಟುಹೋಗಿ.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### ವಿವರಣೆ: ಕೋರ್ ಇಂಪೋರ್ಟ್‌ಗಳು ಮತ್ತು ಮೆಟ್ರಿಕ್ಸ್  
ಕೋರ್ ಲೈಬ್ರರಿಗಳು ಮತ್ತು ರಾಗಾಸ್ ಮೆಟ್ರಿಕ್ಸ್ ಅನ್ನು ಲೋಡ್ ಮಾಡುತ್ತದೆ. ಪ್ರಮುಖ ಅಂಶಗಳು:  
- ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳಿಗೆ SentenceTransformer.  
- `evaluate` + ಆಯ್ದ ರಾಗಾಸ್ ಮೆಟ್ರಿಕ್ಸ್.  
- ಮೌಲ್ಯಮಾಪನ ಕಾರ್ಪಸ್ ರಚನೆಗೆ `Dataset`.  
ಈ ಇಂಪೋರ್ಟ್‌ಗಳು ದೂರಸ್ಥ ಕರೆಗಳನ್ನು ಪ್ರೇರೇಪಿಸುವುದಿಲ್ಲ (ಎಂಬೆಡ್ಡಿಂಗ್‌ಗಳಿಗೆ ಮಾದರಿ ಕ್ಯಾಶ್ ಲೋಡ್ ಹೊರತುಪಡಿಸಿ).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### ವಿವರಣೆ: ಟಾಯ್ ಕಾರ್ಪಸ್ ಮತ್ತು QA ಗ್ರೌಂಡ್ ಟ್ರೂತ್
ಸಣ್ಣ ಇನ್-ಮೆಮೊರಿ ಕಾರ್ಪಸ್ (`DOCS`), ಬಳಕೆದಾರರ ಪ್ರಶ್ನೆಗಳ ಸೆಟ್ ಮತ್ತು ನಿರೀಕ್ಷಿತ ಗ್ರೌಂಡ್ ಟ್ರೂತ್ ಉತ್ತರಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತದೆ. ಇವು ಹೊರಗಿನ ಡೇಟಾ ಪಡೆಯುವಿಕೆ ಇಲ್ಲದೆ ವೇಗವಾಗಿ, ನಿರ್ಧಾರಾತ್ಮಕ ಮೌಲ್ಯಮಾಪನವನ್ನು ಅನುಮತಿಸುತ್ತವೆ. ನಿಜವಾದ ಸಂದರ್ಭಗಳಲ್ಲಿ ನೀವು ಉತ್ಪಾದನಾ ಪ್ರಶ್ನೆಗಳು + ಸಂಗ್ರಹಿತ ಉತ್ತರಗಳನ್ನು ಮಾದರಿಯಾಗಿಸಿಕೊಳ್ಳುತ್ತೀರಿ.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### ವಿವರಣೆ: ಸೇವೆ ಪ್ರಾರಂಭ, ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು ಮತ್ತು ಸುರಕ್ಷತಾ ಪ್ಯಾಚ್  
Foundry Local ನಿರ್ವಾಹಕನನ್ನು ಪ್ರಾರಂಭಿಸುತ್ತದೆ, `promptTemplate` ಗೆ ಸ್ಕೀಮಾ-ಡ್ರಿಫ್ಟ್ ಸುರಕ್ಷತಾ ಪ್ಯಾಚ್ ಅನ್ನು ಅನ್ವಯಿಸುತ್ತದೆ, ಮಾದರಿ ಐಡಿಯನ್ನು ಪರಿಹರಿಸುತ್ತದೆ, OpenAI-ಸಮ್ಮತ ಗ್ರಾಹಕನನ್ನು ರಚಿಸುತ್ತದೆ, ಮತ್ತು ಡಾಕ್ಯುಮೆಂಟ್ ಕಾರ್ಪಸ್‌ಗಾಗಿ ಸಾಂದ್ರ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಪೂರ್ವಗಣನೆ ಮಾಡುತ್ತದೆ. ಇದು ಪುನಃಬಳಕೆ ಮಾಡಬಹುದಾದ ಸ್ಥಿತಿಯನ್ನು ರಿಟ್ರೀವಲ್ + ಜನರೇಶನ್‌ಗಾಗಿ ಸಿದ್ಧಪಡಿಸುತ್ತದೆ.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### ವಿವರಣೆ: ರಿಟ್ರೀವರ್ ಫಂಕ್ಷನ್
ನಾರ್ಮಲೈಜ್ ಮಾಡಿದ ಎಂಬೆಡ್ಡಿಂಗ್‌ಗಳ ಮೇಲೆ ಡಾಟ್ ಪ್ರಾಡಕ್ಟ್ ಬಳಸಿ ಸರಳ ವೆಕ್ಟರ್ ಸಾದೃಶ್ಯ ರಿಟ್ರೀವರ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತದೆ. ಟಾಪ್-k ಡಾಕ್ಸ್ (k=2 ಡೀಫಾಲ್ಟ್) ಅನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ. ಉತ್ಪಾದನೆಯಲ್ಲಿ ಪ್ರಮಾಣ ಮತ್ತು ವಿಳಂಬಕ್ಕಾಗಿ ANN ಸೂಚ್ಯಂಕ (FAISS, Chroma, Milvus) ಜೊತೆಗೆ ಬದಲಾಯಿಸಿ.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### ವಿವರಣೆ: ಜನರೇಶನ್ ಫಂಕ್ಷನ್  
`generate` ಒಂದು ನಿರ್ಬಂಧಿತ ಪ್ರಾಂಪ್ಟ್ ಅನ್ನು ರಚಿಸುತ್ತದೆ (ಸಿಸ್ಟಮ್ ಕೇವಲ ಸಾಂದರ್ಭಿಕವನ್ನು ಬಳಸಲು ಸೂಚಿಸುತ್ತದೆ) ಮತ್ತು ಸ್ಥಳೀಯ ಮಾದರಿಯನ್ನು ಕರೆಸುತ್ತದೆ. ಕಡಿಮೆ ತಾಪಮಾನ (0.1) ಸೃಜನಶೀಲತೆಯ ಮೇಲೆ ನಿಷ್ಠಾವಂತವಾದ ಹೊರತೆಗೆಯುವಿಕೆಯನ್ನು ಪ್ರೋತ್ಸಾಹಿಸುತ್ತದೆ. ಕಡಿತಗೊಳಿಸಿದ ಉತ್ತರ ಪಠ್ಯವನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### ವಿವರಣೆ: ಫಾಲ್ಬ್ಯಾಕ್ ಕ್ಲೈಂಟ್ ಪ್ರಾರಂಭ
ಹಿಂದಿನ ಪ್ರಾರಂಭ ಸೆಲ್ ತಪ್ಪಿಸಿಕೊಂಡಿದ್ದರೂ ಅಥವಾ ವಿಫಲವಾಗಿದ್ದರೂ `client` ಅಸ್ತಿತ್ವದಲ್ಲಿರುತ್ತದೆ ಎಂದು ಖಚಿತಪಡಿಸುತ್ತದೆ—ನಂತರದ ಮೌಲ್ಯಮಾಪನ ಹಂತಗಳಲ್ಲಿ NameError ತಪ್ಪಿಸುತ್ತದೆ.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### ವಿವರಣೆ: ಮೌಲ್ಯಮಾಪನ ಲೂಪ್ ಮತ್ತು ಮೆಟ್ರಿಕ್ಸ್  
ಮೌಲ್ಯಮಾಪನ ಡೇಟಾಸೆಟ್ ಅನ್ನು ನಿರ್ಮಿಸುತ್ತದೆ (ಅಗತ್ಯ ಕಾಲಮ್‌ಗಳು: ಪ್ರಶ್ನೆ, ಉತ್ತರ, ಸಂದರ್ಭಗಳು, ಭೂಮಿತಿಯ ಸತ್ಯಗಳು, ಉಲ್ಲೇಖ) ನಂತರ ಆಯ್ದ ರಾಗಾ ಮೆಟ್ರಿಕ್ಸ್‌ಗಳನ್ನು ಪುನರಾವರ್ತಿಸುತ್ತದೆ.  

ಆಪ್ಟಿಮೈಜೆಷನ್:  
- ಫಾಸ್ಟ್_ಮೋಡ್ ತ್ವರಿತ ಸ್ಮೋಕ್ ಪರೀಕ್ಷೆಗಳಿಗೆ ಉತ್ತರ ಸಂಬಂಧಿತತೆಗೆ ಮಾತ್ರ ಮಿತಿಗೊಳಿಸುತ್ತದೆ.  
- ಪ್ರತಿ ಮೆಟ್ರಿಕ್ ಲೂಪ್ ಒಂದು ಮೆಟ್ರಿಕ್ ವಿಫಲವಾದಾಗ ಸಂಪೂರ್ಣ ಮರುಗಣನೆ ತಪ್ಪಿಸುತ್ತದೆ.  

ಮೆಟ್ರಿಕ್ -> ಅಂಕ (ವಿಫಲವಾದರೆ NaN) ಎಂಬ ಡಿಕ್ಷನರಿ ಅನ್ನು ಔಟ್‌ಪುಟ್ ಮಾಡುತ್ತದೆ.


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
